## Avocado EDA

In [1]:
import pandas as pd
import re
from tqdm import tqdm
tqdm.pandas()  # Register tqdm with pandas

In [2]:
df = pd.read_parquet('../data/avocado_full.parquet')      

In [3]:
df.shape

(938035, 243)

In [4]:
for column in df.columns:
    print(column)

arrival_date
cc_address
create_date
delete_after_submit
delivery_report
file_as
flags
importance
message_cc_me
message_recip_me
message_to_me
messageid
modify_date
original_sensitivity
outlook_recipient_name
outlook_sender_name
priority
processed_subject
read_receipt
recip_access
recip_address
reply_requested
response_requested
sender2_access
sender2_address
sender_access
sender_address
sensitivity
sent_date
sentto_address
subject
body_file_path
body_content
autoforward
outlook_version
in_reply_to
reply_to
Keywords
bcc_address
X-Mailer
X-MimeOLE
X-Received
X-UIDL
return_path_address
X-Juno-Att
X-Juno-Line-Breaks
X-Juno-RefParts
X-LSV-ListID
X-List-Host
X-URL
X-Sender
X-AMAZON-TRACK
X-AMAZON-TRACK-2
X-eGroups-Return
X-Authentication-Warning
X-MIMETrack
X-Accept-Language
X-apix
X-ListID
X-MsgID
X-Att-Encode
X-SohoMail-UID
X-SohoMail-version
X-Listprocessor
X-SubID
X-Server-Uuid
X-WSS-ID
X-To
X-MnM-Date
X-MnM-JobId
X-MnM-MsgId
X-MnM-Software
X-Originating-IP
X-cid
X-Message-Id
X-OriginalA

In [5]:
df.head()

,arrival_date,cc_address,create_date,delete_after_submit,delivery_report,file_as,flags,importance,message_cc_me,message_recip_me,...,X-Authenticated-IP,X-Authenticated-Sender,X-Flags,X-NAV-TimeoutProtection19,X-NAV-TimeoutProtection20,X-NAV-TimeoutProtection21,X-NAV-TimeoutProtection22,X-NAV-TimeoutProtection23,X-NAV-TimeoutProtection24,X-Talk21Ref
0,2001-04-12T22:02:41Z,'ray_rahamin@hp.com'; Dan Baca; Elba Linscott,2001-04-12T23:41:52Z,0.0,0.0,Meshele Ko,1.0,normal,0.0,1.0,...,None,None,NaN,None,None,None,None,None,None,None
1,2000-04-24T23:06:15Z,None,2000-04-26T16:12:23Z,0.0,1.0,All Employees,1.0,normal,0.0,0.0,...,None,None,NaN,None,None,None,None,None,None,None
2,2000-06-20T20:51:04Z,None,2000-06-20T20:34:39Z,0.0,0.0,RisingStar White Paper.doc,17.0,high,0.0,0.0,...,None,None,NaN,None,None,None,None,None,None,None
3,2000-09-12T17:01:08Z,None,2000-09-08T22:48:28Z,0.0,0.0,'jbmcbryde@fedex.com',17.0,normal,0.0,0.0,...,None,None,NaN,None,None,None,None,None,None,None
4,2000-05-03T01:30:54Z,Wayne Thayer,2000-05-08T23:33:53Z,0.0,0.0,Meshele Ko (E-mail),1.0,normal,0.0,1.0,...,None,None,NaN,None,None,None,None,None,None,None


In [6]:
new_cols = ["messageid", "sender_access", "outlook_sender_name", "outlook_recipient_name", "sentto_address", "cc_address", "subject", "processed_subject", "sent_date", "arrival_date", "body_file_path", "body_content", "in_reply_to"   ,  "Keywords",  ]
df_man = df.loc[:, new_cols]

In [7]:
df_man.shape

(938035, 14)

In [8]:
df_man = df_man.dropna(subset=['body_content'])

In [9]:
df_man.shape

(937958, 14)

In [10]:
#df_man.to_parquet('avocado_light.parquet', engine='pyarrow', compression='snappy')

### Filter DF into sender_access: EX / SMTP / NaN

In [11]:
df_ex = df_man[df_man['sender_access'] == 'EX']


#### Sender Type EX

In [12]:
df_ex.shape

(600564, 14)

In [13]:
df_ex['body_content'].str.contains('Message-ID:', case=False).sum()

600564

In [14]:
df_ex.loc[:, 'body_content'] = df_ex['body_content'].fillna('')

# Define the regex pattern
pattern = re.compile(r'message-id:\s*<[^>]+>\s*(.*)', re.IGNORECASE | re.DOTALL)

# Filter DataFrame to get only rows that contain the Message-ID pattern
df_ex = df_ex[df_ex['body_content'].str.contains(r'message-id:\s*<[^>]+>', case=False, regex=True)]

# Extract the text following the Message-ID using .loc to avoid SettingWithCopyWarning
df_ex.loc[:, 'extracted_text'] = df_ex['body_content'].str.extract(pattern, expand=False)


In [15]:
df_ex.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords,extracted_text
0,<FC38A9406AA4D411AB62009027DE9DA5E6545D@HQEXCH01>,EX,Fortunata Hermoso,Meshele Ko,Meshele Ko,'ray_rahamin@hp.com'; Dan Baca; Elba Linscott,Fw: RE: AvocadoIT's CEO committed to winning B...,AvocadoIT's CEO committed to winning Banamex deal,2001-04-12T22:02:41Z,2001-04-12T22:02:41Z,text/162/162-000001-EM.txt,"From: ""Fortunata Hermoso"" <fortunata.hermoso@a...",None,None,Fyi on Venks commitment on Banamex. The next s...
1,<19B3B310D020D311B57E00105A9A55241D5975@COFFEE>,EX,Paiman Komeilizadeh,Meshele Ko,All Employees,None,Expense reimbursement request- Amendment,Expense reimbursement request- Amendment,2000-04-24T23:06:06Z,2000-04-24T23:06:15Z,text/162/162-000003-EM.txt,"From: ""Paiman Komeilizadeh"" <pkomeilizadeh@avo...",None,None,"Dear friends,\r\n\r\n\r\nTo expedite the expe..."
2,<F57EA917353BD411B5BA00D0B708160C121B24@COFFEE>,EX,Meshele Ko,None,Piyush Goel; Dan Baca,None,FW: Information,Information,2000-06-20T20:51:04Z,2000-06-20T20:51:04Z,text/162/162-000004-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None,Here is the information from Send.com. They h...
3,<9640A23C9075D411B5CB00D0B708160C23A93D@COFFEE>,EX,Meshele Ko,None,'jbmcbryde@fedex.com',None,Fedex,Fedex,2000-09-12T17:01:08Z,2000-09-12T17:01:08Z,text/162/162-000005-EM.txt,"From: ""Meshele Ko"" <mko@avocadoit.com>\r\nTo: ...",None,None,"Hello Jim,\r\n\r\nI am following up on a piece..."
5,<19B3B310D020D311B57E00105A9A55243C6C85@COFFEE>,EX,Ron Silverton,Meshele Ko,Marketing&Business Development; Sales Team; Ex...,None,m-commerce over $120 billion by 2008,ASG News & Analysis - 6 April 2000,2000-04-10T19:19:56Z,2000-04-10T19:19:57Z,text/162/162-000008-EM.txt,"From: ""Ron Silverton"" <rsilverton@avocadoit.co...",None,None,Attached are some recent projections from Merr...


##### Extract Original Message

In [16]:
df_ex.loc[0]['extracted_text']

'Fyi on Venks commitment on Banamex. The next step is to set a conference call with Nicollo and Venk next week. EP is committed to do whatever it takes to close this accoiunt.\r\n\r\nWe have a lot of support. Let\'s keep the communcation going. Please use me to escalate any support needed to close this account.\r\n--------------------------\r\nFortunata Hermoso\r\nAvocadoIT, Inc.\r\nGlobal Alliance Manager\r\nMobile Phone: 408.464.8998\r\nemail: fortunata.hermoso@avocadoit.com\r\n\r\n\r\n-----Original Message-----\r\nFrom: Venk Shukla <shukla@avocadoit.com>\r\nTo: \'DEPERALTA,AIREEN (HP-Cupertino,ex1)\' <aireen_deperalta@hp.com>; Venk Shukla <shukla@avocadoit.com>; ARENAS,MARTHA (HP-Mexico,ex1) <martha_arenas@hp.com>; SPATARO,NICCOLO (HP-Mexico,ex1) <niccolo_spataro@hp.com>\r\nCC: KOMORI,ZENKI (HP-Mexico,ex1) <zenki_komori@hp.com>; ZAMORA,EDUARDO (HP-Mexico,ex1) <eduardo_zamora@hp.com>; Fortunata Hermoso <Fortunata.Hermoso@avocadoit.com>; WATSON,JULIE (HP-Cupertino,ex1) <julie_watson@h

In [17]:
# Adjust the search string to capture exactly five dashes, spaces, and words
search_string = r"-{5}[\w\s]+-{5}"

# Regex pattern to match and capture text after the search string
pattern = re.compile(rf'{search_string}[\r\n]*(.*)', re.DOTALL)

# Extract text after the search string into 'preceding_conversation'
df_ex['preceding_conversation'] = df_ex['extracted_text'].str.extract(pattern, expand=False)

# Remove the search string and everything after it from 'extracted_text'
df_ex['extracted_text'] = df_ex['extracted_text'].str.replace(pattern, '', regex=True).str.strip()


In [18]:
df_ex.shape

(600564, 16)

In [19]:
df_ex['extracted_text'].str.contains('In-Reply-To').sum()

240095

In [20]:
# Apply the regex pattern to remove matching lines from 'extracted_text' column
df_ex['extracted_text'] = df_ex['extracted_text'].replace(r'^In-Reply-To: <[^>]*>\s*', '', regex=True)

In [21]:
print(df_ex.iloc[0]['extracted_text'])

Fyi on Venks commitment on Banamex. The next step is to set a conference call with Nicollo and Venk next week. EP is committed to do whatever it takes to close this accoiunt.

We have a lot of support. Let's keep the communcation going. Please use me to escalate any support needed to close this account.
--------------------------
Fortunata Hermoso
AvocadoIT, Inc.
Global Alliance Manager
Mobile Phone: 408.464.8998
email: fortunata.hermoso@avocadoit.com


In [22]:
df_ex['extracted_text'].str.contains('In-Reply-To').sum()

1

In [23]:
pattern = re.compile(
    r'(From:.*|Sent:.*|To:.*|Subject:.*)(?:\n|\r\n|$)', re.IGNORECASE | re.MULTILINE
)

# Extract matching lines into 'preceding_conversation'
df_ex['preceding_conversation'] = df_ex['extracted_text'].str.extract(
    rf'((From:.*\n.*\n.*\n.*\n)(.*))', expand=False
)[0]

# Remove matching lines from 'extracted_text'
df_ex['extracted_text'] = df_ex['extracted_text'].str.replace(pattern, '', regex=True).str.strip()


In [24]:
df_ex.iloc[0]['extracted_text']

"Fyi on Venks commitment on Banamex. The next step is to set a conference call with Nicollo and Venk next week. EP is committed to do whatever it takes to close this accoiunt.\r\n\r\nWe have a lot of support. Let's keep the communcation going. Please use me to escalate any support needed to close this account.\r\n--------------------------\r\nFortunata Hermoso\r\nAvocadoIT, Inc.\r\nGlobal Alliance Manager\r\nMobile Phone: 408.464.8998\r\nemail: fortunata.hermoso@avocadoit.com"

#### Sender Type SMTP

In [430]:
df_smtp = df_man[df_man['sender_access'] == 'SMTP']

In [431]:
df_smtp.shape

(333599, 14)

In [432]:
#pattern = r'(?:(Received:[\s\S]*)(?:--\r?\n\r?\n|Importance:[\s\S]*?\r\n\r\n|"\r\n\r\n|ascii\r\n\r\n|plain\r\n\r\n|format=flowed\r\n\r\n|Content-Disposition: inline\r\n\r\n|V5.00.2314.1300\r\n\r\n|quoted-printable\r\n\r\n)|From[\s\S]*>\r\n\r\n)'
pattern = r'(?:(Received:[\s\S]*)(?:--\r?\n\r?\n|Importance:[\s\S]*?\r\nReturn-Path:[\s\S]*?\r\n\r\n|"\r\n\r\n|ascii\r\n\r\n|plain\r\n\r\n|format=flowed\r\n\r\n|Content-Disposition: inline\r\n\r\n|V5.00.2314.1300\r\n\r\n|quoted-printable\r\n\r\n|Importance:[\s\S]{1,15}\r\n\r\n|\+0000\r\n)|From[\s\S]*>\r\n\r\n)'

df_smtp.loc[:, 'extracted_text'] = df_smtp['body_content'].str.replace(pattern, '', regex=True)

KeyboardInterrupt: 

In [423]:
print(repr(df_smtp.iloc[19]['extracted_text']))

"Received: from dfw-smtpout1.email.verio.net ([129.250.36.41]) by coffee.avocadoit.com with SMTP (Microsoft Exchange Internet Mail Service Version 5.5.2448.0)\r\n\tid HV998H96; Mon, 27 Mar 2000 10:47:36 -0800\r\nReceived: from [129.250.38.63] (helo=dfw-mmp3.email.verio.net)\r\n\tby dfw-smtpout1.email.verio.net with esmtp (Exim 3.12 #7)\r\n\tid 12ZeQy-0002xu-00; Mon, 27 Mar 2000 18:40:16 +0000\r\nReceived: from [207.196.68.136] (helo=spectre)\r\n\tby dfw-mmp3.email.verio.net with smtp (Exim 3.12 #7)\r\n\tid 12ZeQu-0005Yt-00; Mon, 27 Mar 2000 18:40:12 +0000\r\nHi Meshele,\r\n\r\nThanks! I am copying our BizDev guy and our PR contact\r\nto ensure they are up to speed on the demo categories.\r\n\r\nAs for funding...\r\n\r\nWe are going into negotiations over the initial funding\r\noffer this afternoon. We are expecting a second offer\r\nto take in with us. This should be interesting. I'll\r\nlet you know how the progress of the negotiations.\r\n\r\nThis initial round has been absolutely fa

In [414]:
print(df_smtp.iloc[51]['extracted_text'])

Looks good!

I found a couple of things:

1) When I tried signon with an incorrect userid and password, I got "signin again" screen, instead of an error
saying incorrect userid and/or password

2) I'll call you about a situation where one of the items did not have an amount due--it's hard to explain--i'll
call you tonight when i get back to philadelphia

Dave


At 01:05 PM 9/19/00 -0700, Phani Yadavalli wrote:


Hi Dave,
I have fixed the UI and the signout according to your mail.
The PQA that was sent previously would work.
Please go through the application and get back to me with your feed back.
Thank you,
Phani

-----Original Message-----
From: Dave Fortney [ mailto:dfortney@Paytrust.com]
Sent: Friday, September 15, 2000 3:12 PM
To: 'Phani Yadavalli'; Dave Fortney; John Schemena; Jim Eichmann
Cc: Meshele Ko; Darshan Patel; Colin Murphy; Dhaval Vasavada
Subject: RE: Paytrust mobilized XML-feed application is ready for your
rev iew


Phani, THanks for the prompt reply--let's see if thi

In [378]:
print(repr(df_smtp.iloc[51]['body_content']))

'Received: from ptmail01.paytrust.com (dc.paytrust.com [207.239.143.226]) by coffee.avocadoit.com with SMTP (Microsoft Exchange Internet Mail Service Version 5.5.2650.21)\r\n\tid TQ6NQFLC; Mon, 25 Sep 2000 17:22:11 -0700\r\nReceived: from dfortney2.paytrust.com (DFORTNEY2 [10.3.37.222]) by ptmail01.paytrust.com with SMTP (Microsoft Exchange Internet Mail Service Version 5.5.2650.21)\r\n\tid S99R07JM; Mon, 25 Sep 2000 20:08:42 -0400\r\nMessage-Id: <4.3.2.7.2.20000925200150.00b5f7c0@ptmail01.paytrust.com>\r\nX-Sender: dfortney@ptmail01.paytrust.com\r\nX-Mailer: QUALCOMM Windows Eudora Version 4.3.2\r\nDate: Mon, 25 Sep 2000 20:06:28 -0400\r\nTo: Phani Yadavalli <pyadavalli@avocadoit.com>,\r\n Jim Eichmann <jeichmann@Paytrust.com>,Prakash Iyer <PIyer@avocadoit.com>\r\nFrom: Dave Fortney <dfortney@paytrust.com>\r\nSubject: Re: Paytrust mobilized XML-feed application is ready for your\r\n  review\r\nCc: Meshele Ko <mko@avocadoit.com>,Darshan Patel <dpatel@avocadoit.com>,\r\n Colin Murphy <c

#### Sender Type not available

In [30]:
df_nan = df_man[(df_man['sender_access'] != 'SMTP' ) & (df_man['sender_access'] != 'EX') ]

In [31]:
df_nan.shape

(3795, 14)

In [32]:
df_nan.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
20,<F57EA917353BD411B5BA00D0B708160C1745ED@mailho...,None,None,None,None,None,Security,Security,None,2000-08-10T18:48:34Z,text/162/162-000032-EM.txt,Subject: Security\r\nMessage-ID: <F57EA917353B...,None,None
278,<19B3B310D020D311B57E00105A9A55240B4020@COFFEE>,None,None,None,None,None,emulator,emulator,None,2000-05-15T21:36:43Z,text/162/162-000427-EM.txt,Subject: emulator\r\nMessage-ID: <19B3B310D020...,None,None
725,<19B3B310D020D311B57E00105A9A55240B3F4B@COFFEE>,None,None,None,None,None,kbkids,kbkids,None,2000-02-04T18:55:34Z,text/162/162-001121-EM.txt,Subject: kbkids\r\nMessage-ID: <19B3B310D020D3...,None,None
748,<19B3B310D020D311B57E00105A9A55240B3EE7@COFFEE>,None,None,None,None,None,paytrust,paytrust,None,1999-12-07T19:15:04Z,text/162/162-001158-EM.txt,Subject: paytrust\r\nMessage-ID: <19B3B310D020...,None,None
1169,<19B3B310D020D311B57E00105A9A55240B3F00@COFFEE>,None,None,None,None,None,opps,opps,None,1999-12-21T18:33:32Z,text/162/162-001797-EM.txt,Subject: opps\r\nMessage-ID: <19B3B310D020D311...,None,None


### Next Chapter

In [33]:
for column, value in df_man.iloc[0].items():
    print(str(column)+": "+str(value))

messageid: <FC38A9406AA4D411AB62009027DE9DA5E6545D@HQEXCH01>
sender_access: EX
outlook_sender_name: Fortunata Hermoso
outlook_recipient_name: Meshele Ko
sentto_address: Meshele Ko
cc_address: 'ray_rahamin@hp.com'; Dan Baca; Elba Linscott
subject: Fw: RE: AvocadoIT's CEO committed to winning Banamex deal
processed_subject: AvocadoIT's CEO committed to winning Banamex deal
sent_date: 2001-04-12T22:02:41Z
arrival_date: 2001-04-12T22:02:41Z
body_file_path: text/162/162-000001-EM.txt
body_content: From: "Fortunata Hermoso" <fortunata.hermoso@avocadoit.com>
To: "Meshele Ko" <mko@avocadoit.com>
Cc: ray_rahamin@hp.com, "Dan Baca" <dbaca@avocadoit.com>, "Elba Linscott" <elinscott@avocadoit.com>
Subject: Fw: RE: AvocadoIT's CEO committed to winning Banamex deal
Date: 12 Apr 2001 22:02:41 UTC
Message-ID: <FC38A9406AA4D411AB62009027DE9DA5E6545D@HQEXCH01>

Fyi on Venks commitment on Banamex. The next step is to set a conference call with Nicollo and Venk next week. EP is committed to do whatever it

In [34]:
df_man = df_man.sort_values(by="arrival_date")

In [35]:
df_man.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
114827,<FC38A9406AA4D411AB62009027DE9DA50394442D@ntma...,SMTP,rui (Rui Fernandes),None,'ed (Edward Lycklama - KL Group Inc.)','steering',Grab-bag,Grab-bag,1994-07-27T18:46:23Z,1994-07-27T18:46:23Z,text/207/207-001048-EM.txt,From rui Wed Jul 27 14:46:24 1994\nStatus: RO\...,<9407271552.AA20575@klg.com>,None
114676,<FC38A9406AA4D411AB62009027DE9DA503944429@ntma...,SMTP,greg (Greg Kiessling - KL Group Inc.),None,'steering',None,March30 minutes,March30 minutes,1995-03-30T15:41:45Z,1995-03-30T15:41:45Z,text/207/207-000897-EM.txt,From greg Thu Mar 30 10:41:46 1995\nStatus: RO...,None,None
114536,<FC38A9406AA4D411AB62009027DE9DA50394442A@ntma...,SMTP,greg (Greg Kiessling - KL Group Inc.),None,'steering',None,Apr 6 minutes,Apr 6 minutes,1995-04-06T13:20:54Z,1995-04-06T13:20:54Z,text/207/207-000757-EM.txt,From greg Thu Apr 6 09:20:55 1995\nStatus: RO...,None,None
114046,<FC38A9406AA4D411AB62009027DE9DA503942536@ntma...,SMTP,steve (Steve Rosenberg),None,'xrt_dev',None,emacs goody,emacs goody,1995-06-13T21:00:52Z,1995-06-13T21:00:52Z,text/207/207-000265-EM.txt,From steve Tue Jun 13 17:00:54 1995\nStatus: R...,None,None
114391,<FC38A9406AA4D411AB62009027DE9DA50394442B@ntma...,SMTP,greg (Greg Kiessling - KL Group Inc.),None,'steering',None,"CDE, Motif Merge","CDE, Motif Merge",1995-09-12T19:32:57Z,1995-09-12T19:32:57Z,text/207/207-000612-EM.txt,From greg Tue Sep 12 15:32:57 1995\nStatus: RO...,None,None


In [36]:
df_man = df_man.dropna(subset=['body_content'])

In [37]:
df_man.head()

,messageid,sender_access,outlook_sender_name,outlook_recipient_name,sentto_address,cc_address,subject,processed_subject,sent_date,arrival_date,body_file_path,body_content,in_reply_to,Keywords
114827,<FC38A9406AA4D411AB62009027DE9DA50394442D@ntma...,SMTP,rui (Rui Fernandes),None,'ed (Edward Lycklama - KL Group Inc.)','steering',Grab-bag,Grab-bag,1994-07-27T18:46:23Z,1994-07-27T18:46:23Z,text/207/207-001048-EM.txt,From rui Wed Jul 27 14:46:24 1994\nStatus: RO\...,<9407271552.AA20575@klg.com>,None
114676,<FC38A9406AA4D411AB62009027DE9DA503944429@ntma...,SMTP,greg (Greg Kiessling - KL Group Inc.),None,'steering',None,March30 minutes,March30 minutes,1995-03-30T15:41:45Z,1995-03-30T15:41:45Z,text/207/207-000897-EM.txt,From greg Thu Mar 30 10:41:46 1995\nStatus: RO...,None,None
114536,<FC38A9406AA4D411AB62009027DE9DA50394442A@ntma...,SMTP,greg (Greg Kiessling - KL Group Inc.),None,'steering',None,Apr 6 minutes,Apr 6 minutes,1995-04-06T13:20:54Z,1995-04-06T13:20:54Z,text/207/207-000757-EM.txt,From greg Thu Apr 6 09:20:55 1995\nStatus: RO...,None,None
114046,<FC38A9406AA4D411AB62009027DE9DA503942536@ntma...,SMTP,steve (Steve Rosenberg),None,'xrt_dev',None,emacs goody,emacs goody,1995-06-13T21:00:52Z,1995-06-13T21:00:52Z,text/207/207-000265-EM.txt,From steve Tue Jun 13 17:00:54 1995\nStatus: R...,None,None
114391,<FC38A9406AA4D411AB62009027DE9DA50394442B@ntma...,SMTP,greg (Greg Kiessling - KL Group Inc.),None,'steering',None,"CDE, Motif Merge","CDE, Motif Merge",1995-09-12T19:32:57Z,1995-09-12T19:32:57Z,text/207/207-000612-EM.txt,From greg Tue Sep 12 15:32:57 1995\nStatus: RO...,None,None


In [38]:
ctr = 0
def extract_email_text(row):
    global ctr
    if "Message-ID:" in row['body_content']: 
        ctr += 1


In [39]:
df_man.apply(extract_email_text, axis=1)

114827    None
114676    None
114536    None
114046    None
114391    None
          ... 
583076    None
583154    None
583162    None
583172    None
583238    None
Length: 937958, dtype: object

In [40]:
ctr

847441